In [5]:
import ast
import torch
import pandas as pd
import numpy as np
from torch_geometric.data import Data
from sklearn.preprocessing import MinMaxScaler
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch_geometric.nn import global_mean_pool

class GCNWithPooling(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(GCNWithPooling, self).__init__()

        self.conv1 = GCNConv(num_features, 64)
        self.conv2 = GCNConv(64, 128)
        self.conv3 = GCNConv(128, 256)
        self.conv4 = GCNConv(256, 128)
        self.conv5 = GCNConv(128, 64)

        self.pool1 = global_mean_pool
        self.pool2 = global_mean_pool

        self.fc1 = torch.nn.Linear(64, 128)
        self.fc2 = torch.nn.Linear(128, num_classes)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)

        x = self.conv2(x, edge_index)
        x = F.relu(x)

        x = self.conv3(x, edge_index)
        x = F.relu(x)

        x = self.conv4(x, edge_index)
        x = F.relu(x)

        x = self.conv5(x, edge_index)
        x = F.relu(x)

        x = self.pool1(x, batch)
        x = self.pool2(x, batch)

        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = F.relu(x)

        x = self.fc2(x)

        return F.log_softmax(x, dim=1)

csv_path = 'tif_df_train.csv'
df = pd.read_csv(csv_path)

df['Landmarks'] = df['Landmarks'].apply(ast.literal_eval)

emotion_list = ["Angry", "Disgust", "Fear", "Happiness", "Neutral", "Sad", "Surprised"]

def one_hot_encode(emotion, emotions=emotion_list):
    emotion_dict = {emotion: idx for idx, emotion in enumerate(emotions)}
    one_hot_vector = [0.0] * len(emotions)
    if emotion in emotion_dict:
        one_hot_vector[emotion_dict[emotion]] = 1.0
    return one_hot_vector

df['encoded_label'] = df['Labels'].apply(lambda x: one_hot_encode(x))
df['encoded_label'] = df['encoded_label'].apply(lambda x: torch.tensor(x, dtype=torch.float32))

labels = torch.stack([tensor for tensor in df['encoded_label']])

landmarks = df['Landmarks'].apply(lambda x: np.array(x))
landmarks_normalized = np.array([np.array(x) for x in landmarks])
landmarks_reshaped = landmarks_normalized.reshape(-1, 2)

scaler = MinMaxScaler()
landmarks_normalized = scaler.fit_transform(landmarks_reshaped)
landmarks_normalized = landmarks_normalized.reshape(-1, 68, 2)

node_features = torch.tensor(landmarks_normalized, dtype=torch.float32)

def delaunay_triangulation(landmarks):
    from scipy.spatial import Delaunay
    tri = Delaunay(landmarks)
    triangles = tri.simplices
    edges = set()
    for triangle in triangles:
        for i in range(3):
            edge = tuple(sorted([triangle[i], triangle[(i + 1) % 3]]))
            edges.add(edge)
    return triangles, list(edges)

def construct_adjacency_matrix(landmarks):
    _, edges = delaunay_triangulation(landmarks)
    num_landmarks = len(landmarks)
    adjacency_matrix = np.zeros((num_landmarks, num_landmarks), dtype=int)
    for edge in edges:
        i, j = edge
        adjacency_matrix[i, j] = 1
        adjacency_matrix[j, i] = 1
    return adjacency_matrix

adjacency_matrix = construct_adjacency_matrix(landmarks_normalized[0])

edge_index = torch.tensor(np.nonzero(adjacency_matrix), dtype=torch.long)

batch = torch.zeros(node_features.size(0), dtype=torch.long)

data = Data(x=node_features.view(-1, 136), edge_index=edge_index, y=labels, batch=batch)

model = GCNWithPooling(num_features=2, num_classes=7)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.batch)
    loss = loss_fn(out, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

model.eval()
with torch.no_grad():
    out = model(data.x, data.edge_index, data.batch)
    _, predicted = out.max(dim=1)
    print(f"Predicted Classes: {predicted}")

C:\Users\anura\AppData\Local\Temp\ipykernel_14644\1954328406.py:106: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  edge_index = torch.tensor(np.nonzero(adjacency_matrix), dtype=torch.long)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (101x136 and 2x64)

In [7]:
import ast
import torch
import pandas as pd
import numpy as np
from torch_geometric.data import Data
from sklearn.preprocessing import MinMaxScaler
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch_geometric.nn import global_mean_pool
from scipy.spatial import Delaunay

class GCNWithPooling(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(GCNWithPooling, self).__init__()

        self.conv1 = GCNConv(num_features, 64)
        self.conv2 = GCNConv(64, 128)
        self.conv3 = GCNConv(128, 256)
        self.conv4 = GCNConv(256, 128)
        self.conv5 = GCNConv(128, 64)

        self.pool1 = global_mean_pool
        self.pool2 = global_mean_pool

        self.fc1 = torch.nn.Linear(64, 128)
        self.fc2 = torch.nn.Linear(128, num_classes)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)

        x = self.conv2(x, edge_index)
        x = F.relu(x)

        x = self.conv3(x, edge_index)
        x = F.relu(x)

        x = self.conv4(x, edge_index)
        x = F.relu(x)

        x = self.conv5(x, edge_index)
        x = F.relu(x)

        x = self.pool1(x, batch)
        x = self.pool2(x, batch)

        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = F.relu(x)

        x = self.fc2(x)

        return F.log_softmax(x, dim=1)


# Helper function to one-hot encode labels
emotion_list = ["Angry", "Disgust", "Fear", "Happiness", "Neutral", "Sad", "Surprised"]

def one_hot_encode(emotion, emotions=emotion_list):
    emotion_dict = {emotion: idx for idx, emotion in enumerate(emotions)}
    one_hot_vector = [0.0] * len(emotions)
    if emotion in emotion_dict:
        one_hot_vector[emotion_dict[emotion]] = 1.0
    return one_hot_vector

# Load and preprocess CSV data
csv_path = 'tif_df_train.csv'  # Replace with your actual CSV file path
df = pd.read_csv(csv_path)
df['Landmarks'] = df['Landmarks'].apply(ast.literal_eval)

# One-hot encode labels
df['encoded_label'] = df['Labels'].apply(lambda x: one_hot_encode(x))
df['encoded_label'] = df['encoded_label'].apply(lambda x: torch.tensor(x, dtype=torch.float32))

# Convert the list of encoded labels into a tensor
labels = torch.stack([tensor for tensor in df['encoded_label']])

# Process landmarks
landmarks = df['Landmarks'].apply(lambda x: np.array(x))  # Convert landmarks to numpy arrays
landmarks_normalized = np.array([np.array(x) for x in landmarks])  # Convert list of arrays into array

# Normalize the landmarks
landmarks_reshaped = landmarks_normalized.reshape(-1, 2)  # Flatten the 68x2 landmarks into a 2D array
scaler = MinMaxScaler()
landmarks_normalized = scaler.fit_transform(landmarks_reshaped)
landmarks_normalized = landmarks_normalized.reshape(-1, 68, 2)  # Reshape back to (101, 68, 2)

# Prepare node features tensor
node_features = torch.tensor(landmarks_normalized, dtype=torch.float32)

# Delaunay Triangulation and adjacency matrix generation
def delaunay_triangulation(landmarks):
    tri = Delaunay(landmarks)
    triangles = tri.simplices
    edges = set()
    for triangle in triangles:
        for i in range(3):
            edge = tuple(sorted([triangle[i], triangle[(i + 1) % 3]]))
            edges.add(edge)
    return list(edges)

def construct_adjacency_matrix(landmarks):
    edges = delaunay_triangulation(landmarks)
    num_landmarks = len(landmarks)
    adjacency_matrix = np.zeros((num_landmarks, num_landmarks), dtype=int)
    for edge in edges:
        i, j = edge
        adjacency_matrix[i, j] = 1
        adjacency_matrix[j, i] = 1
    return adjacency_matrix

# Generate adjacency matrix (same for all graphs, but could be graph-specific in your case)
adjacency_matrix = construct_adjacency_matrix(landmarks_normalized[0])  # Take the first graph to create the adjacency matrix

# Create edge index
edge_index = torch.tensor(np.nonzero(adjacency_matrix), dtype=torch.long)

# Create batch vector (if you have 101 graphs, the batch size will be 101)
batch = torch.zeros(node_features.size(0), dtype=torch.long)  # One graph at a time, adjust for multiple graphs if needed

# Create Data object for PyTorch Geometric
data = Data(x=node_features.view(-1, 2), edge_index=edge_index, y=labels, batch=batch)

# Initialize the GCN model
model = GCNWithPooling(num_features=2, num_classes=7)  # 2 features (x, y), 7 classes for classification

# Define loss function and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    out = model(data.x, data.edge_index, data.batch)

    # Compute the loss
    loss = loss_fn(out, data.y)

    # Backward pass
    loss.backward()

    # Update the weights
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

# Test the model
model.eval()
with torch.no_grad():
    out = model(data.x, data.edge_index, data.batch)
    _, predicted = out.max(dim=1)
    print(f"Predicted Classes: {predicted}")

RuntimeError: The expanded size of the tensor (6868) must match the existing size (101) at non-singleton dimension 0.  Target sizes: [6868, 64].  Tensor sizes: [101, 1]